### Расчет минимального расстояния между муниципальными районами и метостанциями

**Цели и задачи:**<br>
1. Найти метеостанцию с ближайшим расстоянием к населенному пункту. На основе имеющихся гео-координат метеостанций и населенных пунктов, входящих в муниципальные районы, рассчитать минимальное расстояние между ними (используя формулу гаверсинуса).<br>
2. К набору данных с населенными пунктами добавить id ближайшей метеостанции и сохранить данные в файл "_**settle_ID.csv**_".

#### Загрузка необходимых библиотек

In [1]:
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import numpy as np
import os
import glob
import json
import io

In [2]:
### Формула гаверсинуса, вычисляет расстояние по большой окружности между двумя точками на Земле
def harvesine (lat1, lon1, lat2, lon2):
    # преобразование градусов в радианы
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # формула гаверсинуса
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + sin(dlon/2)**2 * cos(lat1) * cos(lat2)
    c = 2 * asin(sqrt(a))
    r = 6371 # разиус планеты Земля в километрах. Используется значение 3956 миль.
    return c*r 

In [3]:
### Формула для определения метеостанции с минимальным расстоянием
def min_dist(id_dicts, lt1, ln1):
    # создание пустого словаря
    dicto = {}
    # расчет расстяния между населенными пунктами и метеостанциями из словаря
    for i in id_dicts:
        dicto[i] = harvesine (float(lt1), float(ln1), float(id_dicts[i][0]), float(id_dicts[i][1]))
    # найти ключ с наименьшим значением
    best_key = min(dicto, key=dicto.get)
    return best_key

In [4]:
# Чтение данных из файла с информацией о метеостанциях
path = "INPUT/Meteo_link_full.zip"
meteo = pd.read_csv(path, compression='zip', sep=';', header=0, quotechar='"')
meteo = meteo.drop(['Link'], axis=1)
meteo['Latitude'] = meteo['Latitude'].str.replace(",", ".")
meteo['Longitude'] = meteo['Longitude'].str.replace(",", ".")
dicts= meteo.set_index('ID').T.to_dict('list')

In [5]:
# Чтение данных из файла с информацией о населенных пунктах
path = "INPUT/settl_converter.zip"
settl = pd.read_csv(path, compression='zip', sep=';', header=0, quotechar='"')

In [6]:
# Добавление id ближайшей метеостанции
settl['ID'] = settl.apply(lambda row: min_dist(dicts, row.latitude_dd, row.longitude_dd), axis=1)

In [7]:
file = settl.to_csv("INPUT/settle_ID.csv.gz", sep=';', index=False, compression="gzip")